# Importar as bibliotecas

In [68]:
from pyspark.sql.functions import input_file_name, col, lit, regexp_replace, split
from datetime import datetime, timedelta

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# CRIAÇÃO DA VARS EXECUÇÃO

In [12]:
ts_pro = datetime.now().strftime('%Y%m%d')
ts_pro


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'20240801'

# Leituras dos dados no s3

In [74]:
dados = spark.read.option('header','true').csv('s3://auladums3/current_weather_data_portland.csv')
# dados = dados.withColumn('ref', lit(ref))
dados.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-----------------+------------------+-----------------+-----------------+-----------------+--------+-------+-------------------+-------------------+--------------------+-------------------+
|     _c0|      City|      Description|   Temperature (F)|   Feels Like (F)| Minimun Temp (F)| Maximum Temp (F)|Pressure|Humidty|         Wind Speed|     Time of Record|Sunrise (Local Time)|Sunset (Local Time)|
+--------+----------+-----------------+------------------+-----------------+-----------------+-----------------+--------+-------+-------------------+-------------------+--------------------+-------------------+
|       0|  Portland|       few clouds|63.104000000000056|             62.6|           60.242|65.91200000000006|    1019|     74|               0.89|2024-07-18 08:26:15| 2024-07-18 05:39:46|2024-07-18 20:53:47|
|Portland|few clouds|66.48800000000006| 66.12800000000007|63.87800000000007|69.65600000000003|             1019|      70|   2.06|2024-07-18 09:24:13|2024-07

In [75]:
#Contagem da quantidade dos dados que foram extraídos da API
dados.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

25

In [76]:
# Verificação dos Schema da base dados
dados.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Temperature (F): string (nullable = true)
 |-- Feels Like (F): string (nullable = true)
 |-- Minimun Temp (F): string (nullable = true)
 |-- Maximum Temp (F): string (nullable = true)
 |-- Pressure: string (nullable = true)
 |-- Humidty: string (nullable = true)
 |-- Wind Speed: string (nullable = true)
 |-- Time of Record: string (nullable = true)
 |-- Sunrise (Local Time): string (nullable = true)
 |-- Sunset (Local Time): string (nullable = true)

In [77]:
# Validação dos nomes da colunas da base dados

dados.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['_c0', 'City', 'Description', 'Temperature (F)', 'Feels Like (F)', 'Minimun Temp (F)', 'Maximum Temp (F)', 'Pressure', 'Humidty', 'Wind Speed', 'Time of Record', 'Sunrise (Local Time)', 'Sunset (Local Time)']

In [78]:
#Criação da tabela temporária para tratamento dos dados

dados.createOrReplaceTempView('dados')
dados.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[_c0: string, City: string, Description: string, Temperature (F): string, Feels Like (F): string, Minimun Temp (F): string, Maximum Temp (F): string, Pressure: string, Humidty: string, Wind Speed: string, Time of Record: string, Sunrise (Local Time): string, Sunset (Local Time): string]

In [82]:
df_renamed = dados.withColumnRenamed("_c0", "ID")\
                .withColumnRenamed("City", "Cidade") \
               .withColumnRenamed("Description", "Descricao") \
               .withColumnRenamed("Temperature (F)", "Temperatura") \
               .withColumnRenamed("Feels Like (F)", "Aproximadamente") \
               .withColumnRenamed("Minimun Temp (F)", "Temp_Minima") \
               .withColumnRenamed("Maximum Temp (F)", "Temp_Maxima") \
               .withColumnRenamed("Pressure", "Pressao") \
               .withColumnRenamed("Humidty", "Humidade") \
               .withColumnRenamed("Wind Speed", "Velocidade_do_vento") \
               .withColumnRenamed("Time of Record", "Tempo_de_gravacao") \
               .withColumnRenamed("Sunrise (Local Time)", "Nascer_do_sol") \
               .withColumnRenamed("Sunset (Local Time)", "Por_do_Sol")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
df_renamed.show(2)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+-----------------+------------------+-----------------+-----------------+-----------------+-------+--------+-------------------+-------------------+-------------------+-------------------+
|      ID|    Cidade|        Descricao|       Temperatura|  Aproximadamente|      Temp_Minima|      Temp_Maxima|Pressao|Humidade|Velocidade_do_vento|  Tempo_de_gravacao|      Nascer_do_sol|         Por_do_Sol|
+--------+----------+-----------------+------------------+-----------------+-----------------+-----------------+-------+--------+-------------------+-------------------+-------------------+-------------------+
|       0|  Portland|       few clouds|63.104000000000056|             62.6|           60.242|65.91200000000006|   1019|      74|               0.89|2024-07-18 08:26:15|2024-07-18 05:39:46|2024-07-18 20:53:47|
|Portland|few clouds|66.48800000000006| 66.12800000000007|63.87800000000007|69.65600000000003|             1019|     70|    2.06|2024-07-18 09:24:13|2024-07-18 

In [85]:
df_renamed.write.partitionBy().parquet('s3://auladums3/raw/', mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…